In [ ]:
import os
import pickle
import json
import argparse

from tqdm import tqdm
import pandas as pd

python ./preprocess/json_to_tsv.py --load_path data/json/19_150tags_NamedEntity/NXNE2102008030.json --save_path data/raw
python ./preprocess/json_to_tsv.py --load_path data/json/21_150tags_NamedEntity --save_path data/raw

In [ ]:
def define_argparser():
    p = argparse.ArgumentParser()

    p.add_argument(
        '--load_path',
        default = './data/doccano_json',
        help="Path of file or directory to load original corpus."
    )
    p.add_argument(
        '--save_path',
        default = './data/raw',
        help="Path to save data."
    )
    p.add_argument(
        '--return_tsv',
        default = True,
        help="If not true, only pickle file will be saved."
    )

    config = p.parse_args(args=[])

    return config

In [ ]:
config = define_argparser()
config

Namespace(load_path='./data/doccano_json', save_path='./data/raw', return_tsv=True)

In [ ]:
import json
import os

file = os.path.join(config.load_path, '대구지방법원 2007. 10. 2 선고 2007노1818 판결 [폭력행위.jsonl')

data = []
with open(file, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()  # 공백 제거
        if line:  # 빈 줄 무시
            try:
                data.append(json.loads(line))  # 한 줄씩 파싱
            except json.JSONDecodeError as e:
                print(f"Error parsing line: {line}")
                print(f"Error: {e}")

print(data)
print(len(data))

[{'id': 114, 'text': '대구지방법원 2007. 10. 2 선고 2007노1818 판결 [폭력행위등처벌에관한법률위반(집단.흉기등상해) {인정된 죄명 폭력행위등처벌에관한법률위반(집단.흉기등협박)} ]', 'Comments': [], 'label': [[7, 18, 'TI_DATE'], [22, 31, 'CN'], [36, 60, 'NC'], [69, 93, 'NC']]}, {'id': 115, 'text': '재판경과', 'Comments': [], 'label': []}, {'id': 116, 'text': '대법원 2008. 3. 27 선고 2007도8772 판결', 'Comments': [], 'label': [[4, 15, 'TI_DATE'], [19, 28, 'CN']]}, {'id': 117, 'text': '대구지방법원 2007. 10. 2 선고 2007노1818 판결', 'Comments': [], 'label': [[7, 18, 'TI_DATE'], [22, 31, 'CN']]}, {'id': 118, 'text': '전 문', 'Comments': [], 'label': []}, {'id': 119, 'text': '피고인 정철 (820701-1409675)', 'Comments': [], 'label': [[0, 3, 'PS_ASSAILANT'], [4, 6, 'NM']]}, {'id': 120, 'text': '주거 경북 의성군 의성읍 후죽5길 14', 'Comments': [], 'label': [[0, 2, 'LC_RESIDENCE'], [3, 21, 'LC_REGION']]}, {'id': 121, 'text': '본적 경북 의성군 의성읍 경북대로 5690-25', 'Comments': [], 'label': [[0, 2, 'LC_DOMICILE'], [3, 26, 'LC_REGION']]}, {'id': 122, 'text': '항소인 검사', 'Comments': [], 'label': [[0, 3, 'PS_OTHER

In [ ]:
print(data[0]['id'])
print(data[0]['text'])
print(data[0]['Comments'])
print(data[0]['label'])

114
대구지방법원 2007. 10. 2 선고 2007노1818 판결 [폭력행위등처벌에관한법률위반(집단.흉기등상해) {인정된 죄명 폭력행위등처벌에관한법률위반(집단.흉기등협박)} ]
[]
[[7, 18, 'TI_DATE'], [22, 31, 'CN'], [36, 60, 'NC'], [69, 93, 'NC']]


In [ ]:
def json_to_tsv(file: str):
    cols = ['sentence_id', 'sentence', 'ne'] # 데이터 프레임 생성
    df = pd.DataFrame(columns=cols) # 데이터 프레임 생성
    id = 0

    data = []
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()  # 공백 제거
            if line:  # 빈 줄 무시
                try:
                    data.append(json.loads(line))  # 한 줄씩 파싱
                except json.JSONDecodeError as e:
                    print(f"Error parsing line: {line}")
                    print(f"Error: {e}")
    
    ne = []

    for i in range(0, len(data)):
        if data[i]['text'] == None:
            continue
        else:
            df.loc[id, 'sentence'] = data[i]['text']

            # 엔티티 정보를 딕셔너리로 변환
            entity_dict = {}
            for index, (start, end, label) in enumerate(data[i]['label'], 1):
                # 문자열 슬라이싱을 사용하여 형태소 'form' 추출
                form = data[i]['text'][start:end]
                entity_dict[index] = {
                    'form': form,
                    'label': label,
                    'begin': start,
                    'end': end
                }
            # print(entity_dict)
            ne.append(entity_dict)
            id += 1
    df['ne'] = ne

    return df

In [ ]:
load_path = config.load_path # json 파일이 담긴 폴더 혹은 파일 그 자체
fn = os.path.split(config.load_path)[1] # config.load_path에서 파일명만 분리하여 fn 변수에 할당합니다. os.path.split() 함수는 경로를 디렉토리 부분과 파일명 부분으로 나눕니다. 이 중 파일명 부분만을 fn에 저장합니다.
if fn.rfind(".") > 0: # 파일명 fn에서 확장자를 제거합니다. rfind(".")는 파일명에서 마지막으로 나타나는 .의 위치를 찾습니다. 만약 .이 발견되면, 파일명에서 그 위치까지를 잘라서 확장자 없는 파일명을 fn으로 저장합니다.
    fn = fn[:fn.rfind(".")]
save_fn = os.path.join(config.save_path, fn) # 변환된 데이터의 결과물을 저장할 경로를 save_fn 변수에 할당합니다. config.save_path에 지정된 디렉토리 경로와 앞에서 만든 파일명 fn을 합쳐서 저장합니다.

print(f'load_path: {load_path}')
print(f'save_fn: {save_fn}')

load_path: ./data/doccano_json
save_fn: ./data/raw/doccano_json


In [ ]:
if os.path.isdir(load_path): # load_path가 디렉토리인지 확인합니다. 만약 디렉토리라면 그 안에 있는 모든 파일을 반복적으로 처리합니다.
# os.listdir(load_path)는 load_path 디렉토리 안의 모든 파일 목록을 반환합니다.
    dfs = []
    for file in tqdm(os.listdir(load_path)): # tqdm을 사용해 파일 목록을 처리하면서 진행 상태를 시각적으로 표시합니다.
        df = json_to_tsv(os.path.join(load_path, file))
        dfs.append(df)
        # 각 파일을 json_to_tsv() 함수로 읽어들여, 그 결과를 데이터프레임 df로 변환한 뒤 dfs 리스트에 추가합니다.
    data = pd.concat(dfs) # 모든 파일의 데이터프레임을 pd.concat(dfs)로 하나의 데이터프레임 data로 결합합니다.
else:
    data = json_to_tsv(load_path) # 만약 load_path가 파일이라면, 단일 파일을 json_to_tsv() 함수로 처리해 data에 할당합니다.

100%|██████████| 5/5 [00:00<00:00, 98.05it/s]

Error parsing line: {"id":474,"text":"피고인은 2008. 2. 18. 15:40경 대구 달서구 명천로에 있는 화곡병원 307호실에서 그곳에 입원 중인 정우식을 만나고 있던 중, 경북지방경찰청 수사과 마약수사대 소속 경위 최성욱, 경장 김주하가 위 병실로 들어와","Comments":[],"label":[[0,3,"PS_ASSAILANT"],[5,17,"TI_DATE"],[18,24,"TI_TIME"],[25,35,"LC"],[40,44,"LC_OTHER_FACILITY"],[45,50,"LC_OTHER_FACILITY"],[63,66,"NM"],[78,85,"LC"][99,101,"JB"],[102,105,"NM"],[107,109,"JB"],[110,113,"NM"],[117,119,"LC_OTHER_FACILITY"]]}
Error: Expecting ',' delimiter: line 1 column 318 (char 317)


In [ ]:
data

,sentence_id,sentence,ne
0,NaN,대구지방법원 2007. 11. 1 선고 2007고합91 판결 [특정범죄 가중처벌 등...,"{1: {'form': '2007. 11. 1', 'label': 'TI_DATE'..."
1,NaN,참조조문,{}
2,NaN,"특정범죄 가중처벌 등에 관한 법률 제5조의10 제2항전단, 제1항",{}
3,NaN,전 문,{}
4,NaN,"피고인 전수환 (628756-1249682), 노동","{1: {'form': '피고인', 'label': 'PS_ASSAILANT', '..."
...,...,...,...
98,NaN,"한편 피고인은 어린 시절 특수강도죄 등으로 수차례 보호처분을 받은 전력이 있고, 2...","{1: {'form': '피고인', 'label': 'PS_ASSAILANT', '..."
99,NaN,그 후 2007. 1. 30. 가석방되어 2007. 5. 6. 그 잔형기가 경과되었...,"{1: {'form': '2007. 1. 30.', 'label': 'TI_DATE..."
100,NaN,"위와 같은 제반 정상을 모두 종합하여 보면, 피고인을 사회와 상당기간 격리하여 구금...","{1: {'form': '피고인', 'label': 'PS_ASSAILANT', '..."
101,NaN,재판장 판사 백승현 판사 김빛나,"{1: {'form': '백승현', 'label': 'NM', 'begin': 7,..."


# **문장 이어 붙이기 (512 토큰 제한)**

In [ ]:
# 문장을 병합할 기준 개수
sentence_len = 1  

In [ ]:
from transformers import AutoTokenizer
import pandas as pd

# DataFrame 예시 데이터
df = pd.DataFrame(data)

# klue/roberta-base 토크나이저 로드
tokenizer_loader = AutoTokenizer
tokenizer = tokenizer_loader.from_pretrained('klue/roberta-base')

# 각 문장의 토큰 수 계산
df['token_count'] = df['sentence'].apply(
    lambda x: len(tokenizer(x, truncation=False, add_special_tokens=False)["input_ids"])
)

# 병합된 데이터를 저장할 리스트
merged_rows = []
current_sentence = ""
current_ne = {}
current_token_count = 0
current_offset = 0  # 병합된 문장의 현재 길이 (character 단위)
sentence_count = 0  # 병합된 문장 개수 카운터
max_token_length = 0  # 병합된 문장에서 최대 토큰 길이 저장


# 병합 로직
for _, row in df.iterrows():
    # 현재 row의 문장 길이
    sentence_length = len(row['sentence'])

    # 병합 가능 여부 확인 (문장 개수와 토큰 길이)
    if sentence_count < sentence_len and current_token_count + row['token_count'] <= 512:
        # 병합 가능
        current_sentence += ' ' if current_sentence else ''  # 앞 공백 추가 (첫 문장은 제외)
        for key, entity in row['ne'].items():
            # 기존 위치에 현재 문장의 오프셋(문자 기준)을 더해 새로운 위치 계산
            new_key = len(current_ne) + 1
            current_ne[new_key] = {
                'form': entity['form'],
                'label': entity['label'],
                'begin': entity['begin'] + current_offset,
                'end': entity['end'] + current_offset
            }
        # 문장 병합
        current_sentence += row['sentence']
        current_token_count += row['token_count']
        current_offset += sentence_length + 1  # 공백 포함한 길이 추가
        sentence_count += 1
    else:
        # 병합 불가능, 이전 데이터 저장
        merged_rows.append({
            'sentence_id': None,  # 병합된 데이터는 모두 NaN
            'sentence': current_sentence.strip(),
            'ne': current_ne
        })
        # 최대 토큰 길이 갱신
        max_token_length = max(max_token_length, current_token_count)
        # 초기화 후 새로운 데이터 시작
        current_sentence = row['sentence']
        current_ne = {}
        current_token_count = row['token_count']
        current_offset = sentence_length + 1  # 공백 포함한 길이로 초기화
        sentence_count = 1  # 첫 문장은 포함
        for key, entity in row['ne'].items():
            new_key = len(current_ne) + 1
            current_ne[new_key] = {
                'form': entity['form'],
                'label': entity['label'],
                'begin': entity['begin'],
                'end': entity['end']
            }

# 마지막 문장 추가
if current_sentence:
    merged_rows.append({
        'sentence_id': None,  # 병합된 데이터는 모두 NaN
        'sentence': current_sentence.strip(),
        'ne': current_ne
    })
    # 최대 토큰 길이 갱신
    max_token_length = max(max_token_length, current_token_count)

# 병합된 데이터를 데이터프레임으로 변환
merged_df = pd.DataFrame(merged_rows)

# 결과 출력
print(merged_df)
print(f"Maximum token length in merged sentences: {max_token_length}")

# 병합된 데이터 저장
data = merged_df


    sentence_id                                           sentence  \
0          None  대구지방법원 2007. 11. 1 선고 2007고합91 판결 [특정범죄 가중처벌 등...   
1          None                                               참조조문   
2          None               특정범죄 가중처벌 등에 관한 법률 제5조의10 제2항전단, 제1항   
3          None                                                전 문   
4          None                       피고인 전수환 (628756-1249682), 노동   
..          ...                                                ...   
313        None  한편 피고인은 어린 시절 특수강도죄 등으로 수차례 보호처분을 받은 전력이 있고, 2...   
314        None  그 후 2007. 1. 30. 가석방되어 2007. 5. 6. 그 잔형기가 경과되었...   
315        None  위와 같은 제반 정상을 모두 종합하여 보면, 피고인을 사회와 상당기간 격리하여 구금...   
316        None                                  재판장 판사 백승현 판사 김빛나   
317        None                                             판사 박철기   

                                                    ne  
0    {1: {'form': '2007. 11. 1', 'label': 'TI_DATE'...  
1                                            

In [ ]:
# 첫 번째 행의 ne 열 출력
import pprint

pprint.pprint(data.loc[0, 'ne'])


{1: {'begin': 7, 'end': 18, 'form': '2007. 11. 1', 'label': 'TI_DATE'},
 2: {'begin': 22, 'end': 30, 'form': '2007고합91', 'label': 'CN'},
 3: {'begin': 35,
     'end': 63,
     'form': '특정범죄 가중처벌 등에 관한 법률위반(운전자폭행등)',
     'label': 'NC'}}


In [ ]:
label_set = set()

for row in data['ne']:
    for key in row.values():
        label_set.add(key['label'][:2])

label_set

{'AG',
 'BP',
 'CN',
 'DP',
 'EQ',
 'GD',
 'JB',
 'LC',
 'MC',
 'NC',
 'NM',
 'PC',
 'PS',
 'QT',
 'RL',
 'TI'}

In [ ]:
with open(save_fn+'.pickle', "wb") as f: # 변환된 데이터를 피클(pickle) 파일로 저장합니다. 파일명은 save_fn에 .pickle 확장자를 붙인 것입니다. 
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    # pickle.dump() 함수는 데이터를 피클 형식으로 직렬화하고, protocol=pickle.HIGHEST_PROTOCOL을 사용해 가능한 최고의 프로토콜을 사용하여 저장합니다.

if config.return_tsv: # config.return_tsv가 True인 경우, 데이터프레임 data를 TSV 파일로 저장합니다.
    data.to_csv(save_fn+'.tsv', sep='\t', index=False)
    # 파일명은 save_fn에 .tsv 확장자를 붙인 것이며, 탭(\t)으로 구분된 형식으로 저장됩니다.
    # index=False로 설정하여, 데이터프레임의 인덱스는 파일에 포함되지 않도록 합니다.